In [1]:
!pip install langchain faiss-gpu pdfminer-six==20221105 PyMuPDF==1.23.3
!pip install transformers InstructorEmbedding
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
  

In [2]:
import logging as logger
import os.path
from langchain import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from utils.functions import read_pdf_to_string, get_pdf_paths
# from utils.project_config import config
from langchain.embeddings import HuggingFaceInstructEmbeddings


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

if 'config' not in locals():
  config = {}

config['data_dir_path'] = "/content/drive/MyDrive/genai/new_pdfs"

# Default Instructor Model
config['EMBEDDING_MODEL_NAME'] = "hkunlp/instructor-large"  # Uses 1.5 GB of VRAM (High Accuracy with lower VRAM usage)
config["vector_store_path"] = "/content/resources/new_embeddings"
config["vector_store_path"]


'/content/resources/new_embeddings'

In [5]:
# !wget https://docs.nvidia.com/nsight-systems/pdf/UserGuide.pdf

--2023-09-25 11:46:01--  https://docs.nvidia.com/nsight-systems/pdf/UserGuide.pdf
Resolving docs.nvidia.com (docs.nvidia.com)... 23.206.188.170, 23.206.188.159
Connecting to docs.nvidia.com (docs.nvidia.com)|23.206.188.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10948440 (10M) [application/pdf]
Saving to: ‘UserGuide.pdf’

UserGuide.pdf       100%[===================>]  10.44M  22.9MB/s    in 0.5s    

2023-09-25 11:46:02 (22.9 MB/s) - ‘UserGuide.pdf’ saved [10948440/10948440]



In [6]:
# !mv UserGuide.pdf /content/drive/MyDrive/genai/new_pdfs/Nsight_systems/

In [31]:
import logging
import os
import fitz  # PyMuPDF

def get_pdf_paths(dir_path):
    paths = []

    for file_path in os.listdir(dir_path):
        if file_path.endswith(".pdf"):
            doc_path = os.path.join(dir_path, file_path)
            paths.append(doc_path)
        else:
            paths.extend(get_pdf_paths(f"{dir_path}/{file_path}"))
    return paths


def read_pdf_to_string(paths, pdfs_start):
    pdf_to_str = []
    doc_names = []
    for doc_path in paths:
        try:
            doc = fitz.open(doc_path)
            text = ""
            for page_num in range(doc.page_count):
                if page_num > pdfs_start[doc_path]:
                    page = doc[page_num]
                    text += page.get_text()
            pdf_to_str.append(text)
            doc_names.append(doc_path)
        except:
            print(f"Empty PDF: {doc_path}")
    return (pdf_to_str, doc_names)


In [32]:
pdfs_start = {'/content/drive/MyDrive/genai/new_pdfs/cuda/nvJPEG.pdf': 12,
 '/content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_Debugger_API.pdf': 12,
 '/content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_Runtime_API.pdf': 30,
 '/content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_Driver_API.pdf': 32,
 '/content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_C_Best_Practices_Guide.pdf': 12,
 '/content/drive/MyDrive/genai/new_pdfs/Nsight_compute/Profiling_Nsight_compute.pdf': 4,
 '/content/drive/MyDrive/genai/new_pdfs/Nsight_compute/NsightComputeCLI.pdf': 4,
 '/content/drive/MyDrive/genai/new_pdfs/Nsight_compute/NsightCompute.pdf': 4,
 '/content/drive/MyDrive/genai/new_pdfs/Nsight_systems/Nsight_Systems_UserGuide.pdf': 8,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/NVIDIA_Video_Decoder.pdf': 6,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/License.pdf': 2,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/Read_Me.pdf': 2,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/NVENC_Application_Note.pdf': 2,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/NVDEC_Application_Note.pdf': 2,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/NVDEC_VideoDecoder_API_ProgGuide.pdf': 2,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/NVENC_VideoEncoder_API_ProgGuide.pdf': 4,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/NVENC_Preset_Migration_Guide.pdf': 2,
 '/content/drive/MyDrive/genai/new_pdfs/Video_Codec_SDK/Using_FFmpeg_with_NVIDIA_GPU_Hardware_Acceleration.pdf': 2}


In [36]:
print(len(pdfs_start))
pdfs = list(pdfs_start.keys())

18


In [58]:
!rm -rf /content/resources/*

In [59]:

for i in range(0, len(pdfs_start), 2):
    print(f"i = {i}")
    paths = pdfs[i:(i+2)]
    (documents_string, document_names) = read_pdf_to_string(paths, pdfs_start)
    logger.info("Doccuments scanned = " + str(len(documents_string)))
    # TODO plot token graph
    # model.config.max_sequence_length

    logger.info("======Context aware chunking======")
    # Llama 2 supports up to 4096 tokens
    chunk_size = 600
    chunk_overlap = 200
    text_splitter = RecursiveCharacterTextSplitter(separators=[".","\n"], chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    # split text into chunks
    text_content = []
    metadata = []
    for i in range(0, len(documents_string)):
        print(f"Splitting document: {document_names[i]}")
        splits = text_splitter.split_text(documents_string[i])
        text_content.extend(splits)
        metadata.extend([{"file_name": document_names[i]}]*len(splits))

    logger.info("Number of chunks = " + str(len(text_content)))

    logger.info("======Embedding model======")
    embeddings = HuggingFaceInstructEmbeddings(
        model_name=config['EMBEDDING_MODEL_NAME'],
        model_kwargs={"device": "cuda"},
    )

    if not os.path.exists(config['vector_store_path']):
        print("\n\n\n\n Creating embeddings folder")
        os.makedirs(config['vector_store_path'], exist_ok=True)
        print("Creating vector store ")
        vector_store = FAISS.from_texts(
            embedding=embeddings,
            texts=text_content,
            metadatas=metadata
        )
    else:
        vector_store = FAISS.load_local(embeddings=embeddings, folder_path=config['vector_store_path'])
        vector_store.add_texts(text_content)
    print("Saving vector store")
    vector_store.save_local(folder_path=config['vector_store_path'])


i = 0
Splitting document: /content/drive/MyDrive/genai/new_pdfs/cuda/nvJPEG.pdf
Splitting document: /content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_Debugger_API.pdf
load INSTRUCTOR_Transformer
max_seq_length  512




 Creating embeddings folder
Creating vector store 
Saving vector store
i = 2
Splitting document: /content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_Runtime_API.pdf
Splitting document: /content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_Driver_API.pdf
load INSTRUCTOR_Transformer
max_seq_length  512
Saving vector store
i = 4
Splitting document: /content/drive/MyDrive/genai/new_pdfs/cuda/CUDA_C_Best_Practices_Guide.pdf
Splitting document: /content/drive/MyDrive/genai/new_pdfs/Nsight_compute/Profiling_Nsight_compute.pdf
load INSTRUCTOR_Transformer
max_seq_length  512
Saving vector store
i = 6
Splitting document: /content/drive/MyDrive/genai/new_pdfs/Nsight_compute/NsightComputeCLI.pdf
Splitting document: /content/drive/MyDrive/genai/new_pdfs/Nsight_compute/NsightCompute.pdf
load INST

In [60]:
vector_store = FAISS.load_local(embeddings=embeddings, folder_path=config['vector_store_path'])

In [61]:
config['vector_store_path']

'/content/resources/new_embeddings'

In [62]:
!rm -rf /content/drive/MyDrive/genai/new_embeddings
!cp -r /content/resources/* /content/drive/MyDrive/genai/

In [64]:
! zip -r new_embeddings_all.zip /content/resources/new_embeddings

  adding: content/resources/new_embeddings/ (stored 0%)
  adding: content/resources/new_embeddings/index.pkl (deflated 78%)
  adding: content/resources/new_embeddings/index.faiss (deflated 8%)


In [63]:
docs = vector_store.as_retriever(search_kwargs={'k': 10}).get_relevant_documents("Creates a CUDA array according to the CUDA_ARRAY3D_DESCRIPTOR structure pAllocateArray and returns a handle to the new CUDA array in *pHandle. The CUDA_ARRAY3D_DESCRIPTOR is defined as:")

for doc in docs:
  print("\n\n\n\n\n\n\n")
  print(f"{doc.page_content}")









.
Parameters
pHandle
- Returned array
pAllocateArray
- Array descriptor
Returns
CUDA_SUCCESS, CUDA_ERROR_DEINITIALIZED, CUDA_ERROR_NOT_INITIALIZED,
CUDA_ERROR_INVALID_CONTEXT, CUDA_ERROR_INVALID_VALUE,
CUDA_ERROR_OUT_OF_MEMORY, CUDA_ERROR_UNKNOWN
Description
Creates a CUDA array according to the CUDA_ARRAY_DESCRIPTOR structure
pAllocateArray and returns a handle to the new CUDA array in *pHandle








.
CUresult cuArray3DCreate (CUarray *pHandle, const
CUDA_ARRAY3D_DESCRIPTOR *pAllocateArray)
Creates a 3D CUDA array.
Parameters
pHandle
- Returned array
pAllocateArray
- 3D array descriptor
Returns
CUDA_SUCCESS, CUDA_ERROR_DEINITIALIZED, CUDA_ERROR_NOT_INITIALIZED,
CUDA_ERROR_INVALID_CONTEXT, CUDA_ERROR_INVALID_VALUE,
CUDA_ERROR_OUT_OF_MEMORY, CUDA_ERROR_UNKNOWN
Description
Creates a CUDA array according to the CUDA_ARRAY3D_DESCRIPTOR structure
pAllocateArray and returns a handle to the new CUDA array in *pHandle








.
Parameters
pHandle
- Returned mipmapped array
pMipmappe